In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# project_dir = os.path.join('drive','My Drive','Personal Projects','Sleep Apnea Study','apnea-study')
# os.chdir(project_dir)
# !git pull
# !pip install -r colab_requirements.txt 


In [14]:
%load_ext autoreload

import datetime

import tensorflow as tf

import lib.utils as utils
from lib.models import BaseRNN, BaseLSTM, DoubleLSTM
from lib.train import train, evaluate
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Raw Data

In [15]:
utils.create_tfrecords_from_raw_data(raw_data_dir='data/raw_data',tf_rec_data_dir='data/processed_data')

homepap-lab-full-1600042 does not track pulse, skipping
homepap-lab-full-1600043 has sample rate of 64, skipping
homepap-lab-full-1600021 has sample rate of 64, skipping
homepap-lab-full-1600024 has sample rate of 200, skipping
homepap-lab-full-1600031 has sample rate of 2, skipping
homepap-lab-full-1600027 has sample rate of 2, skipping
homepap-lab-full-1600033 has sample rate of 64, skipping
homepap-lab-full-1600003 has sample rate of 200, skipping
homepap-lab-full-1600017 has sample rate of 64, skipping
homepap-lab-full-1600028 does not track pulse, skipping
homepap-lab-full-1600011 has sample rate of 200, skipping
homepap-lab-full-1600004 does not track pulse, skipping
homepap-lab-full-1600013 has sample rate of 64, skipping


### Inputs and Parameters

In [17]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
session_name = 'new_data_test' +  '_' + current_time
optimizer_params = {'learning_rate':0.0001}
training_params = {'num_epochs':100, 'batch_size':512,'apnea_weight':5}
preprocess_params = {'featurize_func' : utils.featurize_2,'seq_len':60,'pulse_sample_rate':16,'data_stride':30}
preprocess_func = utils.preprocess_data
model_params = {'rnn_hidden_dim':20}
test_split = .15
val_split = 0.1765
if tf.test.is_gpu_available():
    print('Using GPU')
    with tf.device("gpu:0"):
        model = BaseLSTM(rnn_hidden_dim=model_params['rnn_hidden_dim'])
        loss_object = tf.keras.losses.BinaryCrossentropy()
        optimizer = tf.keras.optimizers.Adam(optimizer_params['learning_rate'])
else:
    model = BaseLSTM(rnn_hidden_dim=model_params['rnn_hidden_dim'])
    loss_object = tf.keras.losses.BinaryCrossentropy()
    optimizer = tf.keras.optimizers.Adam(optimizer_params['learning_rate'])
log_dir = 'logs/gradient_tape/' + session_name
model_weights_dir = 'model_weights/' + session_name

### Preprocessing and Training

In [10]:
# %load_ext tensorboard
# print(tf.test.is_gpu_available())
# %tensorboard --logdir logs

False

In [18]:
dataset = utils.load_tfrecords(tf_rec_data_dir='data/processed_data')
num_records = 0
for i in dataset:
    num_records += 1

train_data, test_data = utils.split_dataset(dataset, test_split)
train_data, val_data = utils.split_dataset(train_data, val_split)
train_data = preprocess_func(train_data, 
                             featurize_func = preprocess_params['featurize_func'],
                             seq_len=preprocess_params['seq_len'], 
                             pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                             data_stride=preprocess_params['data_stride'])
val_data = preprocess_func(val_data, 
                           featurize_func = preprocess_params['featurize_func'],
                           seq_len=preprocess_params['seq_len'], 
                           pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                           data_stride=preprocess_params['seq_len'])
test_data = preprocess_func(test_data, 
                            featurize_func = preprocess_params['featurize_func'],
                            seq_len=preprocess_params['seq_len'], 
                            pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                            data_stride=preprocess_params['seq_len'])

train_num = train_data._tensors[0].shape[0]
train_bal = train_data._tensors[1].numpy().mean()
val_num = val_data._tensors[0].shape[0]
val_bal = val_data._tensors[1].numpy().mean()
test_num = test_data._tensors[0].shape[0]
test_bal = test_data._tensors[1].numpy().mean()

data_bal = {'train':train_bal,'val':val_bal,'test':test_bal}
data_size = {'train':train_num,'val':val_num,'test':test_num}
print(train_num)
print(train_bal)
print(val_num)
print(val_bal)
print(test_num)
print(test_bal)

30571
0.06763545
2887
0.1353077
3120
0.07686432


In [19]:
train(model, train_data,val_data, loss_object, optimizer, log_dir,model_weights_dir,
      num_epochs=training_params['num_epochs'], 
      batch_size=training_params['batch_size'],
      apnea_weight=training_params['apnea_weight'])
train_res = evaluate(model, train_data)
test_res = evaluate(model, test_data)
print(train_res)
print(test_res)
utils.save_session(session_name,
                   model,
                   model_params,
                   num_records,
                   test_split,
                   val_split,
                   preprocess_func,
                   preprocess_params,
                   data_bal,
                   data_size,
                   training_params,
                   optimizer,
                   optimizer_params,
                   train_res,
                   test_res,
                   log_dir,
                   model_weights_dir,
                   res_path = 'results')

Starting Training
{'accuracy': 0.889909282217352, 'recall': 0.3523992229628973, 'precision': 0.2644635598141696, 'f1': 0.3021636434636265, 'auc': 0.7475163534829683}
{'accuracy': 0.87875, 'recall': 0.3577732990478838, 'precision': 0.27669981187852727, 'f1': 0.31205673758865254, 'auc': 0.7434877843595238}


In [20]:
evaluate(model, val_data)

{'accuracy': 0.8315437016510796,
 'recall': 0.47529652700742386,
 'precision': 0.39754478623938333,
 'f1': 0.43295763699961143,
 'auc': 0.8160623692632343}

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy()

    
for x_batch,y_batch in train_data.batch(128, drop_remainder=True):
    pass
#             with tf.GradientTape() as tape:
#                 predictions = tf.reshape(model(x_batch),[-1])
#                 labels = tf.reshape(y_batch,[-1])
#                 sample_weight = tf.convert_to_tensor(labels.numpy()*apnea_weight)
#                                 loss = loss_object(labels, predictions, sample_weight=sample_weight)


In [ ]:
tf.rank(labels)

In [ ]:
tf.rank(sample_weight)

In [ ]:
tf.squeeze(sample_

In [ ]:
predictions = tf.expand_dims(tf.reshape(model(x_batch),[-1]),1)
labels = tf.expand_dims(tf.reshape(y_batch,[-1]),1)
print(labels.shape)

sample_weight = tf.convert_to_tensor(labels.numpy()*5)
print(sample_weight.shape)
loss = loss_object(labels, predictions, sample_weight=sample_weight)

### Evaluation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lib.utils import to_dense_tensors
%matplotlib inline

dataset = load_tfrecords(tf_rec_data_dir='data/processed_data')
dataset = dataset.map(to_dense_tensors)
for dp in dataset:
    pass
x = dp['x']
y = dp['y']
plt.plot((x - tf.math.reduce_mean(x)) / tf.math.reduce_std(x))
plt.plot(np.repeat(y,16))


In [ ]:
pulse_sample_rate = 16
data_stride = 30
seq_len = 16
X = []
Y = []
#normalize pulse data by night of sleep
x = (x - tf.math.reduce_mean(x)) / tf.math.reduce_std(x)
#convert night of sleep from 1 dimensional sequence of num samples length
#to a sequence of pulse_sample_rate dimension of num seconds length
num_seconds = x.shape[0]//pulse_sample_rate
x_trunc = x[:pulse_sample_rate*(num_seconds)]
x = tf.reshape(x_trunc,[num_seconds,pulse_sample_rate])
#create new datapoints, according to data_stride
num_data_points = (x.shape[0]//data_stride) - (seq_len//data_stride)
for i in range(0,num_data_points):
    X.append(x[data_stride*i:data_stride*i+seq_len])
    Y.append(y[data_stride*i:data_stride*i+seq_len])

In [ ]:
(np.absolute(x[data_stride*i:data_stride*i+seq_len].numpy().flatten()) > 1.5).any()

In [ ]:
type(x[data_stride*i:data_stride*i+seq_len])

In [ ]:
y[data_stride*i:data_stride*i+seq_len].numpy().mean() == 0

### ToDo and Scrap

In [ ]:
#fix learning rate
#visualize function
#develop model search procedure - preprocessing, architecture, training params
#set seeds

#remove outliers
#0-1 scaling 


#write main.py with argparse
#write shell script for parameter tuning
#set up interpretion, eda notebook
#turn model params into keyword args
#add metrics to tensorboard
#add keyboard interupt to training loop
